In [1]:
from __future__ import unicode_literals
from spacy.en import English
nlp = English()

def tokenize(sentence):
    return nlp(unicode(sentence, "utf-8"))

def tok(utf_sent):
    return nlp(sentence)

def unigram_prob(word):
    return nlp.vocab[word].prob


In [2]:
import csv

def addPair(result, word1, word2, score):
    l1 = len(word1)
    l2 = len(word2)

    if l1 < 2 or l2 <2:
        #skipped = skipped +1
        return

    if not word1 in result:
        result[word1] = {word2: score}
        #added = added + 1
    elif not word2 in result[word1]:
        result[word1][word2] = score
        #added = added + 1
    else: result[word1][word2] += score
        
def parseResults():
    results = {}
    with open("data/pairs_clean.csv") as file:
        for line in csv.reader(file):
            addPair(results, line[0], line[1], int(line[2]))
            
    return results

In [3]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

def cleanup(sentence):
    return [i for i in tokenize(sentence) if i not in stop]

In [4]:
from spacy.parts_of_speech import NOUN, VERB

def compareToken(token):
   return token.pos == NOUN or token.pos == VERB

In [5]:
def relation_score(word1, word2):
    if not word1 in relations: return 0
    elif not word2 in relations[word1]: return 0
    else: return relations[word1][word2] / 100

In [16]:
import numpy as np
from scipy import stats
from homonyms import Homonyms

h = Homonyms()
relations = parseResults()
relations["magician"]["hare"] = 437

def relate(sentence):
    tokens = cleanup(sentence)

    found = []
    for token in tokens:
        hom = h.homonyms(token.orth_)
        if any(hom):
            found = [token, hom]

    if not any(found): return 0


    scores = np.zeros(len(found[1][0]))

    for token in tokens:
        if (compareToken(token)):
            i = 0
            for meaning in found[1][0]:
                scores[i] += (relation_score(meaning, token.orth_) + relation_score(token.orth_, meaning)) / np.exp(unigram_prob(token.orth_))
                i = i+1
    return stats.entropy(scores)


In [25]:
score_treshold = 0

In [26]:
puns = open("data/puns.csv").readlines()
scores = []
for pun in puns:
    try:
        scores.append(relate(pun))
    except:
        pass

good_puns = []
i = 0
for score in scores:
    if score > score_treshold:
        good_puns.append(puns[i])
    i = i + 1

In [21]:
non_puns = open("data/non_puns.csv").readlines()
non_scores = []
for non_pun in non_puns:
    try:
        non_scores.append(relate(non_pun))
    except:
        pass

i = 0
better_puns = []
for score in non_scores:
    if score > 0.2:
        better_puns.append(non_puns[i])
    i = i + 1

In [22]:
print len(good_puns)
print len(better_puns)

337
133


In [28]:
better_puns

['His early work had an air of freshness and originality.\n',
 'There is an air of mystery about him.\n',
 'an air drill; an air pump.\n',
 'The station goes off the air at midnight.\n',
 'There is no need to get up in the air over a simple mistake.\n',
 'The audience ate up everything he said.\n',
 'You ought to be home early. We ought to bring her some flowers.\n',
 "He did an awful good job of painting the barn. It's awful hot in here.\n",
 'Eggs used to sell at sixty cents a dozen. This shirt sells for thirty dollars.\n',
 'She was forced to sell up her entire stock of crystal.\n',
 'to send a messenger; They sent their son to college.\n',
 'An off-stage door slam was his cue to enter.\n',
 'securities that earn on a quarterly basis.\n',
 'to elicit the truth; to elicit a response with a question.\n',
 'His coming of age was an epoch in his life.\n',
 "They're much too nice in their dining habits to enjoy an outdoor barbecue.\n",
 'A large bill from the dentist made a hole in her s

NameError: name 'non_scores' is not defined

In [14]:
len(better_puns)

NameError: name 'better_puns' is not defined

In [203]:
tokenize(puns[0])

In [198]:
good_puns

['What did the grape say when it got stepped on? Nothing - but it let out a little whine.\n',
 "Some people's noses and feet are built backwards: their feet smell and their noses run.\n",
 "If you step onto a plane and recognize a friend of yours named Jack don't yell out Hi Jack!\n",
 "The cannibal's cookbook titled 'How to Better Serve your Fellow Man' was written by a guy who had a wife and ate kids.\n",
 'Talking to her about computer hardware I make my mother board.\n',
 "Show me someone in denial and I'll show you a person in Egypt up to their ankles.\n",
 "California is the land of earthquakes and suntans or 'shake and bake'.\n",
 'What is the best store to be in during an earthquake? A stationery store.\n',
 'A restaurant decided to serve submarine sandwiches but later went under.\n',
 'Every so often railroad conductors have to go for retraining.\n',
 'Nurses who give shots do a great job but sometimes get paid a measle-y salary.\n',
 "A family named woods has a large family t

In [213]:
test_set = [
    "A cardboard belt would be a waist of paper.",
    "Doctors are never perfect because they are always practicing and they need a lot of patients.",
    "If you ride a bus you have to pay your fare share.",
    "A dentist with a toothache could have a bad impact on his patients.",
    "When those around King Arthur's table had insomnia, there were a lot of sleepless knights."
]